In [ ]:
# ML-1 Гибридные рекомендательные системы
# Преподаватель: Александр Кондрашкин
#
#    Датасет ml-latest
#    Вспомнить подходы, которые мы разбирали
#    Выбрать понравившийся подход к гибридным
#    системам
#    Написать свою

# Решением будет ссылка на гитхаб с готовым ноутбуком


In [1]:
# 

# 1. если юзер не в базе, то рекомендуем самые популярные фильмы

#
# 2. если юзер в базе
#  2.1. У юзера мало просмотренных фильмов и отзывов. 
#        берем набор фильмов по тегам и подмешиваем самые популярные фильмы 
#  2.2 достаточно фильмов мало отзывов  
#      берем фильмы для близких пользователей по просмотрам и подмешиваем близкие фильмы по тегам
#  2.3 достаточно фильмов достаточно отзывов
#      берем фильмы с прогнозтруемой высокой оценкой
#  Пункт 2.3. продолжает отбор после 2.2. как гибридная система
#
#  п.1 отбор по рейтингу первые 100 фильмов и выборка 20 для первоначальной  рекомендации 
#  п.2.1. TFIDF на тегах (0ю4) смешиваем с TFIDF на жанрах (0.4), плюс часть самых популярных (0.2) 
#         фильмов - просмотренные фильмы   
#  п.2.2. берем близких  
#
#
#
#





In [2]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [14]:
links = pd.read_csv('ml-latest/links.csv')
movies = pd.read_csv('ml-latest/movies.csv')
ratings = pd.read_csv('ml-latest/ratings.csv')
tags = pd.read_csv('ml-latest/tags.csv')

In [4]:
print('Количество фильмов всего:',len(movies['movieId'].unique()))
movies.head()

Количество фильмов всего: 58098


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
print('Количество фильмов c рейтингом:',len(ratings['movieId'].unique()))
ratings.head()

Количество фильмов c рейтингом: 53889


,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [6]:
print('Количество фильмов с тегами:',len(tags['movieId'].unique()))
tags.head()

Количество фильмов с тегами: 45981


,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [7]:
print('Количество фильмов всего:',len(set(ratings['movieId'].unique()) - set(tags['movieId'].unique())))
print('Количество фильмов всего:',len(set(tags['movieId'].unique()) - set(ratings['movieId'].unique())))

Количество фильмов всего: 12117
Количество фильмов всего: 4209


In [8]:
# Для того что бы проверить как работает наша рекомендательная система
# выберем несколько пользователей и уберем для каждого последний фильм из просмотренных и 
# фильм с самым высоким рейтингом (ghb hfdtycndtr htqnbyujd ds,bhftv gjcktlybq gj dhtvtyb)
# для которого поставлен рейтинг (уберем тег если он был поставлен)

# Для эксперимента выберем пользователей с достаточно большим количеством фильмов

test_users = ratings[['userId','movieId']].groupby('userId').count().sort_values('movieId',ascending=False).reset_index().iloc[10:20]
test_users

,userId,movieId
10,158002,5701
11,253511,5356
12,48470,5257
13,183233,5169
14,94843,5130
15,73145,5042
16,37046,5041
17,187986,4951
18,4796,4874
19,236981,4854


In [9]:
# для каждого тестового юзера уберем из таблици с рейтингом последний фильм
last_movie_for_test_user = ratings[ratings['userId'].isin(test_users['userId'])].groupby('userId').apply(lambda x: x[['movieId','timestamp']].sort_values('timestamp',ascending=False)['movieId'].iloc[0])
# для каждого тестового юзера уберем из таблици с рейтингом самый высокорейтинговый фильм 
last_movie_whit_best_ratings_for_test_user = ratings[ratings['userId'].isin(test_users['userId'])].groupby('userId').apply(lambda x: x[['movieId','rating','timestamp']].sort_values(['rating','timestamp'],ascending=False)['movieId'].iloc[0])

In [20]:
last_movie_whit_best_ratings_for_test_user

userId
4796      120625
37046     184997
48470     152173
73145     159817
94843     179119
158002    112552
183233      5617
187986        32
236981    189911
253511      8950
dtype: int64

In [78]:
list_user_movies = []
for i in last_movie_for_test_user.index:   
    list_user_movies.append([i,last_movie_for_test_user[i]])
for i in last_movie_whit_best_ratings_for_test_user.index:   
    list_user_movies.append([i,last_movie_whit_best_ratings_for_test_user[i]])

ratings_train = ratings.copy()     
ratings_train = ratings.copy() 
ratings_train[(ratings_train['userId'] == set(np.array(list_user_movies)[:,0]))]

tags_train = tags.copy()
drop_index = []
for i,m in list_user_movies:
    drop_index += list(ratings_train[((ratings_train['userId'] == i) & (ratings_train['movieId'] == m))].index)
    drop_index+= list(tags_train[(tags_train['userId'] == i) & (tags_train['movieId'] == m)].index)


In [81]:
ratings_train.loc[drop_index]

,userId,movieId,rating,timestamp
470098,4796,174859,3.5,1510429596
3608268,37046,169784,3.5,1525058396
4726676,48470,193323,1.5,1536599183
7128948,73145,193825,1.0,1537734992
9196588,94843,174819,0.5,1533449915
15472019,158002,144982,3.0,1481609038
17929149,183233,5932,4.0,1038246615
18425639,187986,166635,3.5,1523402033
23180999,236981,193615,3.0,1537290490
24807561,253511,85327,2.0,1317692819


In [67]:
len(ratings[ratings['userId'] == 4796ratings['movieId'] != 174859])

/home/user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


27748534

In [57]:
len(ratings) - len(ratings_train)

137211

In [46]:
set(np.array(list_user_movies)[:,0])

{4796, 37046, 48470, 73145, 94843, 158002, 183233, 187986, 236981, 253511}

In [24]:
len(ratings_train)

0

In [22]:
ratings_train

,userId,movieId,rating,timestamp


In [30]:
def change_string(l):
    return '|'.join(str(x) for x in l).replace(' ', '').replace('-', '')

In [32]:
tags_gr = pd.DataFrame(tags.groupby('movieId').apply(lambda x: change_string(list(x['tag']))), columns=['tags']).reset_index()
tags_gr.head()

,movieId,tags
0,1,animated|buddymovie|Cartoon|cgi|comedy|compute...
1,2,fantasy|adaptedfrom:book|animals|badcgi|basedo...
2,3,moldy|old|AnnMargaret|BurgessMeredith|DarylHan...
3,4,characters|girlmovie|characters|chickflick|bas...
4,5,stevemartin|stevemartin|pregnancy|remake|aging...


In [33]:
movie_tag = pd.merge(movies, tags_gr, on='movieId', how='left').fillna('')
movie_tag.head()

,movieId,title,genres,tags
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,animated|buddymovie|Cartoon|cgi|comedy|compute...
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy|adaptedfrom:book|animals|badcgi|basedo...
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy|old|AnnMargaret|BurgessMeredith|DarylHan...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,characters|girlmovie|characters|chickflick|bas...
4,5,Father of the Bride Part II (1995),Comedy,stevemartin|stevemartin|pregnancy|remake|aging...


In [ ]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
# Функция получения фильмов
# Параметры функции
#  user_id
#  tags_query   
#  genres_query
#  movie_id_query
#
# Результат работы функции  
#
#  При запросе в зависимости от параметров получаем список релевантных фильмов
#  
#  порядок поиска фильмов следующий
#  1. получаем список фильмов               
#  
#  
#  
#  


In [40]:
get_movieId_of_user_id(2)

[1186,
 2243,
 2915,
 3363,
 170,
 1962,
 2108,
 1296,
 849,
 2352,
 1235,
 2707,
 2746,
 1244,
 1663]

In [ ]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,3))
X_train_tfidf = tfidf_transformer.fit_transform(X_train['tags_gnr'])

# получим tf_idf для тестовой выборки
X_test_tfidf = tfidf_transformer.transform(X_test['tags_gnr'])


In [ ]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [38]:
def get_movieId_of_user_id(user_id):
    on_tags = set(tags[tags['userId'] == user_id]['movieId'])
    on_ratings = set(ratings[ratings['userId'] == user_id]['movieId'])
    return list(on_tags | on_ratings)



def get_movies(user_id=None, tags_query=None, genres_query=None, movie_id_query=None):
    pass
    #if user_id is None:
    #user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()
    

In [6]:
x = None
if x is None:
    print(x)

None
